### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2019 ###

# Stock Trading Optimization#

#### Jacob Makola (makola@wisc.edu), Finley Myers (fmyers@wisc.edu), Aaron Zhang (zzhang867@wisc.edu), and Lucas Laibly (laibly@wisc.edu)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [General Model](#2.-General)
     1. [Mathematical Model](#9.-Mathematical)
     1. [Solution](#b.-Solution)
     1. [Results](#c.-Results)
1. [Moving Average](#3.-General)
     1. [Mathematical Model](#a.-Mathematical)
     1. [Solution](#b.-Solution)
     1. [Results](#c.-Results)
1. [Regularization](#4.-Results-and-discussion)
    1. [Mathematical Model](#a.-Mathematical)
    1. [Solution](#b.-Solution)
    1. [Results](#c.-Results)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

The first few sentences should give a quick overview of the entire project. Then, elaborate with a description of the problem that will be solved, a brief history (with [citations](https://en.wikipedia.org/wiki/Citation) where appropriate) of how the problem came about, why it's important/interesting, and any other interesting facts you'd like to talk about. You should address and explain where the problem data is coming from (research? the internet? synthetically generated? something you have collected personally, or by surveying your friends?) Also give an outline of the rest of the report.

This section should be 300-600 words long, and **should be accessible to a general audience** (don't assume your reader has taken the class!). Although you should include references, you should also make sure that it is possible to understand the main ideas of the project and the context without clicking on any of the links or reading any of the references. Feel free to include images if you think it'll be helpful:

![fixit flowchart][flow]

For more help on using Markdown, see [this reference](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet).

[flow]: https://s-media-cache-ak0.pinimg.com/736x/f5/75/c5/f575c53b93724808c6f0211890a54900.jpg

## 2. Base Model##

In [8]:
# How much money on hand that we have to work with
initialCash = 100000

using JuMP, Cbc, NamedArrays

# total number of days
days = [1, 2, 3, 4, 5]

# Amazon, Facebook, Mircosoft, Google, Apple -- In Order
stocks = [:am, :f, :m, :g, :ap]

# Data - One week stock value
values = NamedArray(
[1809.07 168.26 118.76 1188.04 190.35
1810.54 170.79 119.02 1184.24 192.54
1823.97 173.56 119.46 1193.36 194.93
1824.93 177.06 120.05 1206.19 195.76
1831.13 176.45 119.99 1214.68 196.57 ], (days, stocks), ("Days", "Stocks"))

m = Model(solver=CbcSolver())

# We can sell stocks
@variable(m, sell[days, stocks] >= 0,Int)

# We buy buy stock
@variable(m, buy[days, stocks] >= 0,Int)

# We can hold our current portfolio from one day to the next
@variable(m, hold[days,stocks] >= 0,Int)

# Inital amount of money at hand to purchase stock
@variable(m, cash[days] >= 0)


@constraint(m, hold_initial[j in stocks], hold[1, j] == buy[1, j])

# The balance of money that we have on hand much be equivalent to the amount we started with
# minue that of what we bought (per value)
@constraint(m, cash[1] == initialCash - sum(values[1, j] * buy[1, j] for j in stocks))

# On our first day we cannot sell anything because we dont have anything to sell
@constraint(m, sell_initial[j in stocks], sell[1, j] == 0)

# How much we buy on our first day must be less than how much money we have started with
@constraint(m, sum(values[1, j] * buy[1, j] for j in stocks) <= initialCash)

# Our hold for the current day must be equal to that of how much we held on our day previously plus how much 
# we bought for the day minus the amount we sold for the day as well. 
@constraint(m, hold_constraint[i in 2:5, j in stocks], hold[i, j] == hold[i - 1, j] + buy[i, j] - sell[i, j])

# To manage how much money we have moving from one day to next
@constraint(m, cash_constraint[i in 2:5], 
                cash[i] == cash[i - 1]  # currrent day's money on hand equal to amount from the previous day
                        + sum(values[i, j] * sell[i, j] for j in stocks) # plus our sold for the day
                        - sum(values[i, j] * buy[i, j] for j in stocks)) # minus how much we bought for the day

# How much we can buy is equal to the cost of the stock we want but must be less than how much money we have
# IMPORTANT : Our model does not supprot gambling (going into debt to buy stock in hopes of it paying off later)
@constraint(m, buy_constraint[i in 2:5], sum(buy[i, j] * values[i, j] for j in stocks) <= cash[i])

# How much we can sell depends on how much we are currently holding in our portfolio.
# We cannot sell what we do not own
@constraint(m, sell_constraint[i in 2:5, j in stocks], sell[i, j] <= hold[i - 1, j])

# On the last day, we sell everything we are holding for maximum profit in a singular trading week
@constraint(m, sum(hold[5, j] for j in stocks) == 0)

@objective(m, Max, cash[5])
solve(m)

returnVal = getobjectivevalue(m)
println("Portfolio return after one week ", returnVal)
# For some quick tips on using $\LaTeX$, 
# see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).


Portfolio return after one week 105594.70000000001


## 3. Moving Average##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **We will be running your code**. Having multiple code blocks separated by text blocks that explain the various parts of your solution will make it much easier for us to understand your project. You may also solve several versions of your problem with different models/assumptions. **Remember that if you do not write your description of the project and commeent your code well, we cannot understand what you have done. Even if it is technically brilliant, you will loose points if you do not write well and comment your code well.**

It's fine to call external packages such as `Gurobi`, but try to minimize the use of other packages. We want to be able to understand what is happening in your code without looking up additional references.
## a. Mathematical Model ##

## b. Solution ##


## c. Results ##

In [1]:
# this is a code block
using JuMP, Clp
m = Model(solver = ClpSolver())

things = [:horses, :donkeys, :goats]  # these are the things 
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
solve(m)

for i in things
    println("The total number of ", i, " is: ", getvalue(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


Remember to make sure your code compiles! I will be running your code!

## 4. Regularization ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |


## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.